<a href="https://colab.research.google.com/github/audrb1999/DL-Project/blob/main/01.numpy_XOR_%EA%B5%AC%ED%98%84/04.Sigmoid%EC%99%80%20MSE%20%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##***시그모이드와 MSE을 만들어보자***



In [1]:
# 이전코드

import numpy as np

class Layer:
    def __init__(self):# 모든 함수들이 공통으로 입출력을 조절
        self.input = None
        self.output = None
    
    def forward(self, x):# 모든 함수들이 공통으로 사용하는 순전파 
        self.input = x
        self.output = self._forward(x)
        return self.output

    def _forward(self, x):
        raise NotImplementedError

class StepFunction(Layer):
    def _forward(self, x):
        y = x > 0
        return y.astype(np.int)

class Perceptron(Layer):
    def __init__(self, weights, bias, activation=None):
        
        super().__init__() # 만약 상위 클래스 Layer가 호출되지 않아도 실행할 수 있게하는 것
        self.weights = weights
        self.bias = bias
        self.activation = activation

    def _forward(self, x): # 활성화 함수 유무를 위한 if 문
        a = np.matmul(x, self.weights) + self.bias
        if self.activation != None:
            y = self.activation.forward(a)
            return y
        return a

class Model(Layer):
    def __init__(self):
        super().__init__()
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)


    def _forward(self, x):
        a = self.layers[0].forward(x)# 2층으로 연결해주기 위해서 이 출력을 다음 레이어가 받음
        c = self.layers[1].forward(x)
        y = []
        for i in range(len(a)):    
            z = Perceptron(np.array([0.3, 0.3]), -0.5, StepFunction()).forward(np.array([a[i], c[i]]))
            y.append(z)
            
        return y

제대로 동작하는지 확인

In [ ]:
s = StepFunction()
model = Model()
model.add(Perceptron(np.array([0.3, 0.3]), -0.2, activation = s))# OR
model.add(Perceptron(np.array([-0.5, -0.5]), 0.8, s))# nand
model.forward(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]))

[0, 1, 1, 0]

##***Sigmoid를 만들어보자!***

In [2]:
class Sigmoid(Layer):
    def _forward(self, x):
        return 1 / (1+np.exp(-x))

In [3]:
s = Sigmoid()
s.forward(np.array([-2, -1, 0, 1, 2]))

array([0.11920292, 0.26894142, 0.5       , 0.73105858, 0.88079708])

##***MSE(Loss function)을 만들어보자***

In [5]:
class Loss:
    def forward(self, y_true, y_pred):
        raise NotImplementedError
   
    def _forward(self, y_true, y_pred):
        raise NotImplementedError

class MSE(Loss):
    def forward(self, y_true, y_pred):# forward를 상속받아서 고쳐쓴 것
        mse = np.mean(np.square(y_true - y_pred)/2, axis = 0, keepdims = True)# 1랭크가 되지 않도록 차원 유지!
        return mse

In [6]:
mse = MSE()
mse.forward(np.array([1, 2, 3, 4]), np.array([0, 0, 0, 0]))

array([3.75])